## 进阶RAG检索   Contextual Compression + Filtering  

### 前期准备

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma


# Load pdf
loader = PyPDFLoader("..\\..\\baichuan.pdf")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data[:6])

import os
from getpass import getpass

OPENAI_API_KEY = getpass()

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# VectorDB
embedding = OpenAIEmbeddings(base_url="...")
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

retriever = vectordb.as_retriever()

In [4]:
base_docs = retriever.get_relevant_documents(
    "What is baichuan2 ？"
)

base_docs

C:\Users\kalin\AppData\Local\Temp\ipykernel_10256\1791937861.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  base_docs = retriever.get_relevant_documents(


[Document(metadata={'author': '', 'creationdate': '2025-04-18T00:32:55+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2025-04-18T00:32:55+00:00', 'page': 1, 'page_label': '2', 'producer': 'pdfTeX-1.40.25', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': '..\\..\\baichuan.pdf', 'subject': '', 'title': '', 'total_pages': 28, 'trapped': '/False'}, page_content='evaluations, Baichuan 2 nearly doubles the results\nof the Baichuan 1. In addition, Baichuan 2 also\ndemonstrates strong performance on medical and\nlegal domain tasks. On benchmarks such as\nMedQA (Jin et al., 2021) and JEC-QA (Zhong\net al., 2020), Baichuan 2 outperforms other open-\nsource models, making it a suitable foundation\nmodel for domain-specific optimization.\nAdditionally, we also released two chat\nmodels, Baichuan 2-7B-Chat and Baichuan 2-'),
 Document(metadata={'author': '', 'creationdate': '2025-04-18T00:32:55+00:00', 'cr

#### Contextual Extractor

In [6]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, base_url="...")
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "What is baichuan2 ？"
)

In [7]:
compressed_docs

[Document(metadata={'author': '', 'creationdate': '2025-04-18T00:32:55+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2025-04-18T00:32:55+00:00', 'page': 1, 'page_label': '2', 'producer': 'pdfTeX-1.40.25', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': '..\\..\\baichuan.pdf', 'subject': '', 'title': '', 'total_pages': 28, 'trapped': '/False'}, page_content='Baichuan 2 nearly doubles the results\nof the Baichuan 1. In addition, Baichuan 2 also\ndemonstrates strong performance on medical and\nlegal domain tasks. On benchmarks such as\nMedQA (Jin et al., 2021) and JEC-QA (Zhong\net al., 2020), Baichuan 2 outperforms other open-\nsource models, making it a suitable foundation\nmodel for domain-specific optimization.'),
 Document(metadata={'author': '', 'creationdate': '2025-04-18T00:32:55+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2025-04-18T00:32:55+00:00', 'page': 1, 

#### Contextual filters

##### LLMChainFilter

In [8]:
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "What is baichuan2 ？"
)

compressed_docs

[Document(metadata={'author': '', 'creationdate': '2025-04-18T00:32:55+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2025-04-18T00:32:55+00:00', 'page': 1, 'page_label': '2', 'producer': 'pdfTeX-1.40.25', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': '..\\..\\baichuan.pdf', 'subject': '', 'title': '', 'total_pages': 28, 'trapped': '/False'}, page_content='evaluations, Baichuan 2 nearly doubles the results\nof the Baichuan 1. In addition, Baichuan 2 also\ndemonstrates strong performance on medical and\nlegal domain tasks. On benchmarks such as\nMedQA (Jin et al., 2021) and JEC-QA (Zhong\net al., 2020), Baichuan 2 outperforms other open-\nsource models, making it a suitable foundation\nmodel for domain-specific optimization.\nAdditionally, we also released two chat\nmodels, Baichuan 2-7B-Chat and Baichuan 2-'),
 Document(metadata={'author': '', 'creationdate': '2025-04-18T00:32:55+00:00', 'cr

##### EmbeddingsFilter

In [9]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

embeddings_filter = EmbeddingsFilter(embeddings=embedding, similarity_threshold=0.76)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "What is baichuan2 ？"
)

In [ ]:
compressed_docs